In [ ]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../')
from src.EspnFantasyLeague import EspnFantasyLeague
from src.utils.io_json import read_json
from src.utils.utils import filter_by_round_stats, fantasy_team_ranking_per_stat
from src.utils.utils import h2h_score_matrix, aggregate_h2h_scores

# Input variables

In [ ]:
week = 8  # Change this to round/week of the fantasy league that you want the analysis for.
scoring_period = 55  # change this to the day of the fantasy league since the start. This is roughly 7*`week`.
# Warning: If you get a matchup period, see message starting with "Processing matchup period:" below other than that specified in the `week` above, 
# reconsider the `scoring_period` variable again.

# Fetch Data

In [ ]:
settings = read_json('../config/config.json')

In [ ]:
cookies = settings['cookies']
league_settings = settings['league']
url_settings = {
    'url_fantasy': settings['url']['fantasy_league'],
    'url_nba': settings['url']['nba']
}

In [ ]:
espn_api = EspnFantasyLeague(cookies=cookies,
                             **url_settings,
                             **league_settings)
stats = espn_api.fantasy_stats

In [ ]:
teams_data = espn_api.get_teams_data()

In [ ]:
stats_per_round = espn_api.get_fantasy_team_stats_per_round(teams_data)

In [ ]:
stats_latest_round = filter_by_round_stats(stats_per_round, week=week, stats=stats)

In [ ]:
mins_games_df = espn_api.get_adv_stats_per_fantasy_team(endpoints=['mMatchup', 'mMatchupScore'], 
                                                        scoring_period=scoring_period)

# Stats of the round per Fantasy Team

In [ ]:
stats_mins_games_latest_round = stats_latest_round.merge(mins_games_df, left_index=True, right_index=True)

In [ ]:
stats_mins_games_latest_round.rename(index=espn_api.team_abbr_name_dict)

## Ranking index for each statistical category

In [ ]:
stats_ranking_latest_round = fantasy_team_ranking_per_stat(stats_mins_games_latest_round)

In [ ]:
stats_ranking_latest_round.rename(index=espn_api.team_abbr_name_dict)

# Head to Head scores for all possible match-ups in the specified round

In [ ]:
score_matrix_df = h2h_score_matrix(stats_latest_round)

In [ ]:
aggr_round_scores = aggregate_h2h_scores(score_matrix_df)

In [ ]:
score_matrix_df

## Percentage wins from the H2H matchups

In [ ]:
aggr_round_scores.rename(index=espn_api.team_abbr_name_dict).sort_values('Wins', ascending=False)